In [132]:
import pandas as pd
import openpyxl
PATH_TO_EXCEL = "/Users/prashantgaurav/Projects/KaajAi/receipts/Bank-Statement-Template-1-TemplateLab.xlsx"



In [133]:
file = pd.read_excel(PATH_TO_EXCEL).fillna('').to_string(index=False)

In [134]:
file

'              Unnamed: 0                                                                    Unnamed: 1 Unnamed: 2 Unnamed: 3              Unnamed: 4           Unnamed: 5 Unnamed: 6 Unnamed: 7         Unnamed: 8\n                         231 Valley Farms Street\\nSanta Monica, CA 90403\\nfirstcitizensbank@domain.com                                                                                                            \n                                                                                                                                                                                                                  \n                                                                                                                                                     STATEMENT OF ACCOUNT                                         \n                                                                                                                                                     

In [135]:
import openai
import os
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))

prompt = """I'm going to give you the contents of a pdf of a bank statement. The text containes transaction records along with other things.
you job is to parse each transaction record and return the details like date, description, amount and type. The type column should contain either 
'credit' or 'debit'. Return these information in a tabular format. Do not return anything else just the table"""

docs = [{"role": "system", "content": prompt + '\n' + file}]
response = client.chat.completions.create(
                model="gpt-4",
                messages=docs
            )

In [136]:
resp = response.choices[0].message.content

In [137]:
print(resp)

|      Date     |                  Description                  |   Debit   |  Credit  |   Type  |
|---------------|-----------------------------------------------|-----------|----------|---------|
|   05/26/2024  |         Payment - Credit Card                 |    5400   |    -     |  Debit  |
|   05/21/2024  |         Payment - Insurance                   |    3000   |    -     |  Debit  |
|   05/18/2024  |         Account Transfer In                   |     -     |  500000  |  Credit |
|   05/16/2024  |         Cheque Deposite                       |     -     |   10000  |  Credit |
| 2024-09-05    |         Payment - Electricity                 |    1500   |    -     |  Debit  |
| 2024-03-05    |         Payment - Water Utility               |     600   |    -     |  Debit  |
| 2024-03-05    |         Payment - Car Loan                    |    3500   |    -     |  Debit  |
| 2024-01-05    |         Account Transfer Out                  |   80000   |    -     |  Debit  |


In [138]:
def parse_response_table(table):
    lines = resp.strip().split("\n")
    table = [line.split('|') for line in lines]
    final_table = []
    for line in table:
        new_line = []
        for ele in line:
            ele = ele.replace('-','')
            if ele :
               new_line.append(ele.strip())
        if not new_line:
            continue
        final_table.append(new_line)
    return final_table
x = parse_response_table(table)

In [139]:
x

[['Date', 'Description', 'Debit', 'Credit', 'Type'],
 ['05/26/2024', 'Payment  Credit Card', '5400', '', 'Debit'],
 ['05/21/2024', 'Payment  Insurance', '3000', '', 'Debit'],
 ['05/18/2024', 'Account Transfer In', '', '500000', 'Credit'],
 ['05/16/2024', 'Cheque Deposite', '', '10000', 'Credit'],
 ['20240905', 'Payment  Electricity', '1500', '', 'Debit'],
 ['20240305', 'Payment  Water Utility', '600', '', 'Debit'],
 ['20240305', 'Payment  Car Loan', '3500', '', 'Debit'],
 ['20240105', 'Account Transfer Out', '80000', '', 'Debit']]

In [140]:
df = pd.DataFrame(x[1:],columns=x[0])

In [141]:
df

,Date,Description,Debit,Credit,Type
0,05/26/2024,Payment Credit Card,5400,,Debit
1,05/21/2024,Payment Insurance,3000,,Debit
2,05/18/2024,Account Transfer In,,500000,Credit
3,05/16/2024,Cheque Deposite,,10000,Credit
4,20240905,Payment Electricity,1500,,Debit
5,20240305,Payment Water Utility,600,,Debit
6,20240305,Payment Car Loan,3500,,Debit
7,20240105,Account Transfer Out,80000,,Debit
